In [1]:
from FundData import *
from helper import Helper
import pprint, json
import fitz

dir_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
#dir_path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"

dry_path = r'\output\DryRun.pdf'
fin_path = r'\files\financial_indices.xlsx'
fund_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\Dec 24"


In [2]:
mutual_fund = Helper.get_fund_paths(fund_path)

In [ ]:
""" SAMCO PDF FILE MAIN CODE"""

object = Samco(dir_path, dry_path, fin_path)
samco_path = mutual_fund['Samco Mutual Fund']

fund_data = object.fund_data
path, imp, fund_titles = object.check_and_highlight(samco_path, fund_data)

sample = input("Do you want to proceed (Y/N)?: ")

pages =  [3, 5, 7, 9, 11, 13, 15, 17, 18]
bbox = object.content_bbox
data_cond = object.data_cond

data = object.get_clipped_data(samco_path,pages, bbox, fund_titles)
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond)
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

#CREATE FINAL CONTENT    
final_text = dict()
for fund, items in extracted_text.items():
    
    content_dict = dict()
    for header, content in items.items():
        header_content = object.match_regex_to_content(header, content)
        content_dict.update(header_content)

    final_text[fund] = content_dict

In [ ]:
""" TATA PDF FILE MAIN CODE"""

object = Tata(dir_path, dry_path, fin_path)
file_path = mutual_fund['Tata Mutual Fund']

fund_data = object.fund_data
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)

sample = input("Do you want to proceed (Y/N)?: ")

pages =  [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83]
bbox = object.content_bbox
data_cond = object.data_conditions

data = object.get_clipped_data(file_path,pages, bbox, fund_titles)
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond)
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)


In [4]:
""" FRANKLIN TEMPLETON FILE MAIN CODE"""

object = FranklinTempleton(dir_path, dry_path, fin_path)
file_path = mutual_fund['Franklin Templeton Mutual Fund']

fund_data = object.fund_data
path, imp, fund_titles = object.check_and_highlight(file_path, fund_data)

sample = input("Do you want to proceed (Y/N)?: ")



Doc saved at: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\files\pdf_report.xlsx

Pages to extract: [17, 18, 20, 21, 22, 25, 26, 27, 28, 29, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50]


In [5]:
pages =  [17, 18, 20, 21, 22, 25, 26, 27, 28, 29, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 49, 50]
bbox = object.content_bbox
data_cond = object.data_conditions

data = object.get_clipped_data(file_path,pages, bbox, fund_titles)
data = object.extract_span_data(data,[])
clean_data = object.process_text_data(data, data_cond)
nested_data, matrix = object.create_nested_dict(clean_data,20.0, 10.0)
extracted_text = object.get_generated_content(nested_data, object.DRYPATH)

In [15]:
extracted_text

{'franklin india multi cap fund': {'investment_objective': ['The investment objective of the scheme is',
   'to generate long term capital appreciation',
   'by investing in a portfolio of equity and',
   'equity related securities of large cap,',
   'midcap and small cap companies.',
   'DATE OF ALLOTMENT',
   'July 29, 2024',
   'FUND MANAGER(S)',
   'R. Janakiraman, Kiran Sebastian,',
   'Akhil Kalluri &',
   'Sandeep Manam',
   '(dedicated for making investments for Foreign Securities)',
   'BENCHMARK',
   'NIFTY 500 Multicap 50:25:25 TRI',
   'NAV AS OF DECEMBER 31, 2024',
   'Growth Plan 10.1199',
   'IDCW Plan 10.1199',
   'Direct - Growth Plan 10.1868',
   'Direct - IDCW Plan 10.1868',
   'As per the addendum dated March 31, 2021, the',
   'Dividend Plan has been renamed to Income',
   'Distribution cum capital withdrawal (IDCW) Plan',
   'with effect from April 1, 2021',
   'FUND SIZE (AUM)',
   'Month End 4674.98 crores',
   'Monthly Average 4687.63 crores',
   'TURNOVER',
  

In [ ]:
all_blocks[17]['blocks']